In [8]:
!pip install -q pandas

ERROR: snowflake-ml-python 1.0.10 has requirement pandas<2,>=1.0.0, but you'll have pandas 2.2.1 which is incompatible.
ERROR: snowflake-ml-python 1.0.10 has requirement snowflake-snowpark-python<2,>=1.5.1, but you'll have snowflake-snowpark-python 1.5.0 which is incompatible.
ERROR: snowflake-connector-python 3.7.1 has requirement urllib3<2.0.0,>=1.21.1; python_version < "3.10", but you'll have urllib3 2.1.0 which is incompatible.
ERROR: botocore 1.34.51 has requirement urllib3<1.27,>=1.25.4; python_version < "3.10", but you'll have urllib3 2.1.0 which is incompatible.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [7]:
!pip install "snowflake-snowpark-python[pandas]"

     |████████████████████████████████| 419kB 5.8MB/s eta 0:00:01
     |████████████████████████████████| 829kB 83.0MB/s eta 0:00:01
     |████████████████████████████████| 2.6MB 52.2MB/s eta 0:00:01
     |████████████████████████████████| 71kB 19.5MB/s eta 0:00:01
     |████████████████████████████████| 747kB 73.3MB/s eta 0:00:01
     |████████████████████████████████| 163kB 85.9MB/s eta 0:00:01
     |████████████████████████████████| 61kB 21.3MB/s eta 0:00:01
     |████████████████████████████████| 112kB 92.4MB/s eta 0:00:01
     |████████████████████████████████| 143kB 85.5MB/s eta 0:00:01
     |████████████████████████████████| 450kB 81.7MB/s eta 0:00:01
     |████████████████████████████████| 71kB 22.8MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/9c/3d/a121f284241f08268b21359bd425f7d4825cffc5ac5cd0e1b3d82ffd2b10/pytz-2024.1-py2.py3-none-any.whl
     |████████████████████████████████| 153kB 82.1MB/s eta 0:00:01
     |████████████████████████████████| 4.6MB

In [2]:
from snowflake.snowpark.session import Session
import pandas as pd
import subprocess

In [3]:
from snowflake.snowpark.session import Session

connection_params = {
    'user': 'MANISH',
    'password': 'Password@2023',
    'account': 'ug94937.us-east4.gcp',
    'warehouse': 'FOSFOR_FDC',
    'database': 'FDC_DATA_MANISH',
    'schema': 'PUBLIC',
    'role': 'MANISH'
}
session = Session.builder.configs(connection_params).create()
session.sql('use warehouse FOSFOR_FDC;').collect()
session.sql('use database FDC_DATA_MANISH;').collect()
session.sql('use schema FDC_DATA_MANISH.PUBLIC;').collect()

df =  pd.read_csv("/notebooks/notebooks/customer_data.csv")
df_model=session.createDataFrame(
        df.values.tolist(),
        schema=df.columns.tolist())
df_model.write.mode("overwrite").save_as_table("FDC_DATA_MANISH.PUBLIC.CC_CUSTOMER_DATA")

SnowparkSQLException: (1304): 01b3863c-0000-5b48-0000-576d05e93af2: 002086 (22000): SQL compilation error:
Invalid bind value (nan) for type (REAL).